In [25]:
import duckdb
import pandas as pd
import os
import numpy as np

In [2]:
DATA_FOLDER = os.path.join(os.getcwd(), '../data/processed/')

In [3]:
# Use file-based DB since data is large.
con = duckdb.connect(database='./nlp_final.duckdb', read_only=False)

In [13]:
con.execute("CREATE TABLE IF NOT EXISTS ratings_text_cleaned_lemma AS SELECT * FROM read_parquet('" + DATA_FOLDER + "ratings_text_cleaned_lemma.pq')")

In [4]:
con.execute("SELECT * FROM ratings_text_cleaned_lemma LIMIT 5").fetchdf()

,text,__index_level_0__,text_cleaned,text_cleaned_lemma
0,"Puszka 0,33l dzięki Christoph . Kolor jasnozło...",0,puszka 033l dzięki christoph kolor jasnozłoty...,puszka 033l dzięki christoph kolor jasnozłot...
1,Cerveza pale lager gabonesa. MÃ¡s floja que la...,1,cerveza pale lager gabonesa mãs floja que la r...,cerveza pale lager gabonés mãs flojo regabarom...
2,"Kolor- złoty, klarowny. Piana - drobna, średni...",2,kolor złoty klarowny piana drobna średnio wys...,kolor złoty klarowny piać drobny średnio wys...
3,"Botella, de GabÃ³n regalo familiar.31/01/2015C...",3,botella de gabãn regalo familiar31012015color ...,botella gabãn regalo familiar31012015color ama...
4,Many thanks for this beer to Erzengel. Pours l...,4,many thanks for this beer to erzengel pours li...,thank beer erzengel pour light golden medium h...


In [7]:
con.execute("CREATE TABLE IF NOT EXISTS ratings_no_text AS SELECT * FROM read_parquet('" + DATA_FOLDER + "ratings_no_text.pq')")

In [12]:
# Concatenates text reviews with metadata.
con.execute("""
CREATE TABLE IF NOT EXISTS ratings AS
WITH ratings_text_with_rownum AS (
    SELECT *, ROW_NUMBER() OVER () AS rownum
    FROM ratings_text_cleaned_lemma
),
ratings_no_text_with_rownum AS (
    SELECT *, ROW_NUMBER() OVER () AS rownum
    FROM ratings_no_text
)
SELECT
    r1.*,
    r2.*
FROM ratings_text_with_rownum r1
JOIN ratings_no_text_with_rownum r2
ON r1.rownum = r2.rownum;
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### Convert embeddings to format that enables quick analysis

In [3]:
embeddings = np.load(DATA_FOLDER + 'embeddings.npy')

In [4]:
embeddings.shape

(7102520, 384)

In [5]:
# Moves to pandas so its 2D
df = pd.DataFrame(embeddings)

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.092251,0.128006,0.167581,0.093993,0.003538,-0.025684,0.156834,-0.206852,-0.043540,0.126090,...,-0.125435,-0.205509,0.089621,0.020336,0.058301,0.115385,0.095444,-0.117911,0.304145,0.099468
1,-0.040249,0.227907,0.073693,0.042528,0.050313,-0.257208,0.370077,0.086232,0.058457,0.104833,...,-0.019627,-0.219118,-0.189044,0.084599,-0.172757,-0.109665,0.181105,-0.023650,0.043990,-0.097015
2,0.029447,0.064680,0.141761,0.163236,-0.066617,-0.107245,0.421406,-0.139581,0.056566,-0.053757,...,-0.139586,-0.082740,0.044802,0.026649,0.032082,0.058311,0.228115,-0.001229,0.219331,-0.006441
3,-0.367951,0.356342,-0.076709,0.064348,0.396878,-0.301984,0.482078,-0.158974,0.086975,-0.073683,...,-0.051616,-0.245416,-0.284482,0.024146,0.148042,-0.288214,0.315874,-0.008937,0.074141,-0.079015
4,-0.112088,0.086483,0.258012,0.142859,-0.212467,-0.187502,0.416985,0.102876,-0.153799,0.128701,...,-0.201743,-0.194152,0.142829,0.089229,-0.284410,-0.132212,0.179270,-0.068468,0.335503,-0.083845


In [7]:
df.to_parquet(DATA_FOLDER + 'embeddings.pq')

In [16]:
# Move embeddings into database.
con.execute("CREATE TABLE IF NOT EXISTS embeddings_raw AS SELECT * FROM read_parquet('" + DATA_FOLDER + "embeddings.pq')")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
con.execute("SELECT * FROM embeddings_raw LIMIT 5").fetchdf()

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.092251,0.128006,0.167581,0.093993,0.003538,-0.025684,0.156834,-0.206852,-0.043540,0.126090,...,-0.125435,-0.205509,0.089621,0.020336,0.058301,0.115385,0.095444,-0.117911,0.304145,0.099468
1,-0.040249,0.227907,0.073693,0.042528,0.050313,-0.257208,0.370077,0.086232,0.058457,0.104833,...,-0.019627,-0.219118,-0.189044,0.084599,-0.172757,-0.109665,0.181105,-0.023650,0.043990,-0.097015
2,0.029447,0.064680,0.141761,0.163236,-0.066617,-0.107245,0.421406,-0.139581,0.056566,-0.053757,...,-0.139586,-0.082740,0.044802,0.026649,0.032082,0.058311,0.228115,-0.001229,0.219331,-0.006441
3,-0.367951,0.356342,-0.076709,0.064348,0.396878,-0.301984,0.482078,-0.158974,0.086975,-0.073683,...,-0.051616,-0.245416,-0.284482,0.024146,0.148042,-0.288214,0.315874,-0.008937,0.074141,-0.079015
4,-0.112088,0.086483,0.258012,0.142859,-0.212467,-0.187502,0.416985,0.102876,-0.153799,0.128701,...,-0.201743,-0.194152,0.142829,0.089229,-0.284410,-0.132212,0.179270,-0.068468,0.335503,-0.083845


## Data generation for the website

In [12]:
DATA_OUT = os.path.join(os.getcwd(), '../data/out/')

In [13]:
con.execute("SELECT * FROM ratings LIMIT 5").fetchdf().keys()

Index(['text', '__index_level_0__', 'text_cleaned', 'text_cleaned_lemma',
       'rownum', 'date', 'beer_id', 'user_id', 'brewery_id', 'abv', 'style',
       'rating', 'palate', 'taste', 'appearance', 'aroma', 'overall', 'year',
       'brewery_name', 'country_brewery', 'state_brewery', 'country_user',
       'state_user', '__index_level_0___1', 'rownum_1', 'embeddings'],
      dtype='object')

In [14]:
# Get all countries.
countries = con.execute("SELECT DISTINCT ratings.country_user FROM ratings").fetchdf()['country_user'].tolist()

In [15]:
# Create a new folder for every country.
# NOTE: This function has to be identical in JS.
def countriaze(country):
    return country.replace(' ', '_').replace('/', '_').replace('(', '').replace(')', '').replace(',', '').replace('.', '').replace("'", '').replace('"', '').replace('?', '').replace('!', '').replace(':', '').replace(';', '').replace('-', '_').replace('&', 'and').replace('___', '_').replace('__', '_').lower().strip()

for country in countries:
    if country is None:
        continue
    os.makedirs(DATA_OUT + countriaze(country), exist_ok=True)

In [51]:
con.execute("""SELECT * FROM ratings WHERE country_user == 'Japan' LIMIT 5""").fetchdf()

,text,__index_level_0__,text_cleaned,text_cleaned_lemma,rownum,date,beer_id,user_id,brewery_id,abv,...,overall,year,brewery_name,country_brewery,state_brewery,country_user,state_user,__index_level_0___1,rownum_1,embeddings
0,"Very peppery, hoppy, very hop forward and ling...",1443,very peppery hoppy very hop forward and linger...,peppery hoppy hop forward linger hoppy finishn...,1444,2011-06-12 12:00:00,130299,4953,2099,5.3,...,14.0,2011,Hilden,United Kingdom,None,Japan,None,1443,1444,NaN
1,Light golden color. Grainy cereal nose. Fairly...,1813,light golden color grainy cereal nose fairly m...,light golden color grainy cereal nose fairly m...,1814,2011-06-12 12:00:00,92714,4953,2099,4.2,...,11.0,2011,Hilden,United Kingdom,None,Japan,None,1813,1814,NaN
2,Kinda boring average English bitter with not a...,2562,kinda boring average english bitter with not a...,kinda boring average english bitter lot go ok,2563,2011-06-12 12:00:00,31973,4953,4347,4.5,...,11.0,2011,Whitewater,United Kingdom,None,Japan,None,2562,2563,NaN
3,"Lots of roasty character, a touch sour with so...",2672,lots of roasty character a touch sour with som...,lot roasty character touch sour chocolate note...,2673,2011-07-28 12:00:00,123611,4953,4347,4.5,...,14.0,2011,Whitewater,United Kingdom,None,Japan,None,2672,2673,NaN
4,"Light mildly sour, corn and grain. Nothing spe...",2724,light mildly sour corn and grain nothing speci...,light mildly sour corn grain special step aver...,2725,2011-06-12 12:00:00,85648,4953,4347,4.5,...,12.0,2011,Whitewater,United Kingdom,None,Japan,None,2724,2725,NaN


### Top 25 most frequently used words per country

In [49]:
con.execute("""
DROP TABLE IF EXISTS country_user_words;
CREATE TABLE country_user_words AS
WITH tokenized AS (
    SELECT
        country_user,
        UNNEST(STRING_SPLIT(text_cleaned_lemma, ' ')) AS token
    FROM ratings WHERE country_user IS NOT NULL
),
filtered_tokens AS (
    SELECT
        country_user,
        token
    FROM tokenized
    WHERE token != ''
),
word_counts AS (
    SELECT
        country_user,
        token AS word,
        COUNT(*) AS word_count
    FROM filtered_tokens
    GROUP BY country_user, word
),
sorted_word_counts AS (
    SELECT
        country_user,
        word,
        word_count
    FROM word_counts
    ORDER BY country_user, word_count DESC, word
),
result AS (
    SELECT
        country_user,
        LIST(word ORDER BY word_count DESC, word) AS words,
        LIST(word_count ORDER BY word_count DESC, word) AS frequencies
    FROM sorted_word_counts
    GROUP BY country_user
)
SELECT * FROM result;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [50]:
con.execute("SELECT * FROM country_user_words").fetchdf()

,country_user,words,frequencies
0,Virgin Islands (British),"[great, super, abit, brew, gold, label, skol, ...","[4, 4, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, ..."
1,Barbados,"[beer, good, caribbean, ability, aroma, balanc...","[3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Botswana,"[beer, aroma, taste, great, commercial, enjoy,...","[9, 5, 5, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, ..."
3,Jersey,"[beer, good, hop, taste, aroma, hoppy, bitter,...","[91, 50, 30, 30, 26, 24, 23, 23, 22, 19, 19, 1..."
4,Kyrgyzstan,"[good, acceptable, alcohol, appearance, cigare...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...
165,Dem Rep of Congo,"[beer, well, drink, good, have, heavy, taste, ...","[5, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, ..."
166,Togo,"[beer, brew, asia, catfish, disappoint, manito...","[4, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, ..."
167,Trinidad and Tobago,"[bottle, shandy, 1, itâs, taste, thatâs, think...","[3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, ..."
168,United States,"[head, aroma, flavor, pour, light, malt, beer,...","[2185467, 1874702, 1676043, 1594773, 1508633, ..."


In [22]:
for country in countries:
    if country is None:
        continue
    top = con.execute("SELECT words, frequencies FROM country_user_words WHERE country_user = '" + country + "'").fetchdf()
    country = countriaze(country)
    top25 = top['words'].tolist()[0]
    top25 = top25[:min(25, len(top25))]

    top25_freqs = top['frequencies'].tolist()[0]
    top25_freqs = top25_freqs[:min(25, len(top25))]

    top25 = [f"{word} {freq}" for word, freq in zip(top25, top25_freqs)]

    top25 = '\n'.join(top25)
    with open(DATA_OUT + country + '/top25.txt', 'w') as f:
        f.write(top25)

### Brewery data analysis

In [7]:
# Calculates average ratings and counts for international and local stats.
con.execute("""
DROP TABLE IF EXISTS country_user_breweries;
CREATE TABLE country_user_breweries AS
-- International stats
WITH brewery_stats AS (
    SELECT
        country_brewery,
        brewery_name,
        AVG(rating) AS avg_rating,
        COUNT(*) AS num_ratings
    FROM ratings
    GROUP BY
        country_brewery,
        brewery_name
),
national_stats AS (
    SELECT
        r.country_brewery,
        r.brewery_name,
        AVG(r.rating) AS avg_national_rating,
        COUNT(*) AS num_national_ratings
    FROM ratings r
    WHERE r.country_user = r.country_brewery
    GROUP BY
        r.country_brewery,
        r.brewery_name
)
SELECT
    bs.country_brewery,
    bs.brewery_name,
    ROUND(bs.avg_rating, 2) AS avg_rating,
    bs.num_ratings,
    ROUND(COALESCE(ns.avg_national_rating, 0), 2) AS avg_national_rating,
    COALESCE(ns.num_national_ratings, 0) AS num_national_ratings
FROM brewery_stats bs
LEFT JOIN national_stats ns
    ON bs.country_brewery = ns.country_brewery
    AND bs.brewery_name = ns.brewery_name
ORDER BY
    bs.country_brewery,
    bs.avg_rating DESC;
""")

In [8]:
con.execute("SELECT * FROM country_user_breweries LIMIT 10").fetchdf()

,country_brewery,brewery_name,avg_rating,num_ratings,avg_national_rating,num_national_ratings
0,Abkhazia,Stara Praha,2.07,10,0.0,0
1,Abkhazia,Pivovarenniy Zavod Sukhumskiy,1.83,16,0.0,0
2,Abkhazia,Abkhazian Drinks Company,1.35,2,0.0,0
3,Albania,Beer Houz Puka,3.50,1,0.0,0
4,Albania,Albion Brauhaus 1.0 - Kaltenbeer,3.29,9,0.0,0
5,Albania,Shoqeria Picana,3.20,2,0.0,0
6,Albania,Brauhaus Shallvare Tirana,2.98,36,0.0,0
7,Albania,Hotel Martini,2.90,1,0.0,0
8,Albania,Svejk Beer Garden,2.34,7,0.0,0
9,Albania,Birra Korça,2.22,192,0.0,0


In [ ]:
for country in countries:
    if country is None:
        continue
    top = con.execute("SELECT brewery_name, avg_rating, avg_national_rating, num_ratings, num_national_ratings FROM country_user_breweries WHERE country_brewery = '" + country + "' ORDER BY avg_rating DESC").fetchdf()

    country = countriaze(country)
    values = []
    for i, row in top.iterrows():
        values.append({
            'brewery': row['brewery_name'],
            'rating': row['avg_rating'],
            'rating_n': row['num_ratings'],
            'national_rating': row['avg_national_rating'],
            'national_rating_n': row['num_national_ratings']
        })

    if len(values) == 0:
        continue

    # print(country)
    # print(values)
    # Create JSON files since its easier for JS to load.
    with open(DATA_OUT + countriaze(country) + '/breweries.json', 'w') as f:
        f.write(pd.DataFrame(values).to_json(orient='records'))

### Embeddings analysis

In [42]:
# Concat ratings with the embeddings generated by the model.
con.execute("""
CREATE OR REPLACE VIEW ratings_and_embeddings AS
WITH ratings_rn AS (
    SELECT
        country_user,
        text_cleaned_lemma,
        text,
        ROW_NUMBER() OVER () AS rownum
    FROM ratings
),
embeddings_rn AS (
    SELECT *,
     ROW_NUMBER() OVER () AS rownum
    FROM embeddings_raw
)
SELECT
    r.*,
    e.*
FROM ratings_rn r
JOIN embeddings_rn e
ON r.rownum = e.rownum;
""")


In [60]:
con.execute("SELECT * FROM ratings_and_embeddings LIMIT 10").fetchdf()

,country_user,text_cleaned_lemma,text,rownum,0,1,2,3,4,5,...,375,376,377,378,379,380,381,382,383,rownum_1
0,United Kingdom,cask bell inn nottingham irish ales festival h...,Cask at Bell Inn (Nottingham) Irish Ales Festi...,1,-0.092251,0.128006,0.167581,0.093993,0.003538,-0.025684,...,-0.205509,0.089621,0.020336,0.058301,0.115385,0.095444,-0.117911,0.304145,0.099468,1
1,Czech Republic,1401bottle bit hazy golden ochre body medium w...,"#1401Bottle. A bit hazy golden to ochre body, ...",2,-0.040249,0.227907,0.073693,0.042528,0.050313,-0.257208,...,-0.219118,-0.189044,0.084599,-0.172757,-0.109665,0.181105,-0.023650,0.043990,-0.097015,2
2,Czech Republic,bottle 05l clear gold color aroma sweet spicy ...,"Bottle 0.5l. Almost clear, gold colored. AROMA...",3,0.029447,0.064680,0.141761,0.163236,-0.066617,-0.107245,...,-0.082740,0.044802,0.026649,0.032082,0.058311,0.228115,-0.001229,0.219331,-0.006441,3
3,United Kingdom,cask praf 2017 pour clear golden nose lemongra...,"Cask at praf 2017. Pours clear golden, nose is...",4,-0.367951,0.356342,-0.076709,0.064348,0.396878,-0.301984,...,-0.245416,-0.284482,0.024146,0.148042,-0.288214,0.315874,-0.008937,0.074141,-0.079015,4
4,United Kingdom,bottle ninaâs mini marketappearance open bel...,Bottle from Ninaâs Mini Market.Appearance - ...,5,-0.112088,0.086483,0.258012,0.142859,-0.212467,-0.187502,...,-0.194152,0.142829,0.089229,-0.284410,-0.132212,0.179270,-0.068468,0.335503,-0.083845,5
5,None,bready wheat aroma lemon flavour little cloudy...,Bready wheat aroma with lemon flavours. A litt...,6,-0.014657,0.107616,0.240158,-0.027481,-0.382556,-0.073134,...,0.005449,0.153774,-0.111205,0.051713,0.079943,-0.048213,-0.006159,0.213269,0.035907,6
6,Ireland,pour cloudy golden thick white foamy head lemo...,Pours cloudy golden with thick white foamy hea...,7,-0.036262,-0.155591,0.169877,0.150383,-0.180703,0.004156,...,-0.305479,-0.027269,0.210100,-0.328638,-0.201859,0.148948,0.211395,0.299113,-0.089174,7
7,United Kingdom,cask glasgow real ale festival interesting bee...,Cask at the Glasgow Real Ale Festival. This is...,8,-0.002692,0.057585,0.079342,0.072236,-0.130476,-0.140585,...,-0.366486,-0.053217,0.144561,-0.048134,-0.157159,0.151059,0.116193,0.329540,-0.130829,8
8,United States,bottle baggot street wines golden body medium ...,Bottle from Baggot Street Wines. Golden body w...,9,-0.171729,0.006385,0.165901,0.057083,-0.122326,-0.017356,...,-0.029181,0.067784,0.196352,-0.089466,-0.037035,-0.028901,-0.148990,0.122991,-0.140807,9
9,United States,pour slightly hazy golden color white head ton...,Pours a slightly hazy golden color with white ...,10,-0.031614,0.060221,0.129679,0.272514,0.124291,-0.397986,...,-0.485956,-0.213766,0.223312,-0.046067,-0.132272,0.064701,0.152010,0.652622,-0.016794,10


In [63]:
# Calculates average of embeddings per country. Stores table to reduce memory overhead.
statement = """
DROP TABLE IF EXISTS country_avg_embeddings;
CREATE TABLE country_avg_embeddings AS (
    SELECT
        country_user,"""
for i in range(384):
    statement += f"""
        AVG("{i}") AS avg_{i},"""

statement += """
    FROM ratings_and_embeddings
    GROUP BY country_user
)
"""
con.execute(statement)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [66]:
# Example.
con.execute("SELECT * FROM country_avg_embeddings WHERE country_user = 'Greece'").fetchdf()

,country_user,avg_0,avg_1,avg_2,avg_3,avg_4,avg_5,avg_6,avg_7,avg_8,...,avg_374,avg_375,avg_376,avg_377,avg_378,avg_379,avg_380,avg_381,avg_382,avg_383
0,Greece,-0.025542,-0.011353,0.138322,0.089685,-0.006037,-0.032329,0.322234,-0.038377,-0.117951,...,-0.129439,-0.112458,0.038556,0.029317,-0.062721,-0.002304,0.139319,-0.005533,0.268423,-0.003563


In [4]:
def cross(table1, table2):
    """
    :param table1: 
    :param table2: 
    :return: Returns SQL statement for Σ(tb1_avg * tb2_avg)
    """

    st = "SUM("
    for i in range(384):
        st += f"{table1}.avg_{i} * {table2}.avg_{i} + "
    return st[:-3] + ")"

# self-crossing
def scross(table):
    return cross(table, table)

### Cosine Similarity Calculation

The formula for **cosine similarity** between two vectors **A** and **B** is:

$$\text{Cosine Similarity} = \cos(\theta) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|}$$


In [ ]:
statement = f"""
DROP TABLE IF EXISTS country_similarities;
CREATE TABLE country_similarities AS (
    SELECT
        c1.country_user as country1,
        c2.country_user as country2,
        -- Cosine similarity calculation
        (
            {cross('c1', 'c2')}
        ) / (
            SQRT({scross('c1')}) *
            SQRT({scross('c2')})
        ) as similarity
    FROM country_avg_embeddings c1
    CROSS JOIN country_avg_embeddings c2
    WHERE c1.country_user != c2.country_user
    GROUP BY c1.country_user, c2.country_user
)
"""

con.execute(statement)

In [86]:
con.execute("SELECT * FROM country_similarities where country1 == 'Greece' ORDER BY similarity DESC LIMIT 100").fetchdf()

,country1,country2,similarity
0,Greece,Romania,0.999757
1,Greece,Israel,0.999737
2,Greece,Portugal,0.999630
3,Greece,Thailand,0.999619
4,Greece,Iceland,0.999595
...,...,...,...
95,Greece,Laos,0.971084
96,Greece,Kazakhstan,0.970385
97,Greece,Namibia,0.969373
98,Greece,Montenegro,0.968921


In [83]:
con.execute("SELECT * FROM country_similarities where country1 == 'Saint Helena' ORDER BY similarity DESC LIMIT 100").fetchdf()


,country1,country2,similarity
0,Saint Helena,Poland,0.997885
1,Saint Helena,Greenland,0.997359
2,Saint Helena,Belarus,0.984858
3,Saint Helena,Norfolk Island,0.984609
4,Saint Helena,Bulgaria,0.983912
...,...,...,...
95,Saint Helena,Kazakhstan,0.951657
96,Saint Helena,Algeria,0.951073
97,Saint Helena,Laos,0.948962
98,Saint Helena,Mauritius,0.945282


In [87]:
# Calculates, for each country, the most similar countries in terms of how they rate.
con.execute("""
CREATE OR REPLACE VIEW top_similar_countries AS (
    SELECT
        country1,
        country2,
        similarity,
        ROW_NUMBER() OVER (PARTITION BY country1 ORDER BY similarity DESC) as rn
    FROM country_similarities
)
""")

In [5]:
# Calculates the similarity between reviews to each country's average rating.
statement = """
CREATE OR REPLACE VIEW review_similarities AS (
    SELECT
        j.country_user,
        j.text_cleaned_lemma,
        j.text,
        -- Cosine similarity between review embedding and country average
        (
            SUM(
                ("""
for i in range(384):
    statement += f"""j."{i}" * c.avg_{i} + """

statement = statement[:-3] + f""")
            ) / (
                SQRT(SUM(
    """
for i in range(384):
    statement += f"""j."{i}" * j."{i}" + """

statement = statement[:-3] + """)) *
                SQRT(""" + scross('c') + """)
            )
        ) as similarity
    FROM ratings_and_embeddings j
    JOIN country_avg_embeddings c ON j.country_user = c.country_user
    GROUP BY j.country_user, j.text_cleaned_lemma, j.text
)
"""

In [89]:
con.execute(statement)

In [90]:
con.execute("SELECT * FROM review_similarities LIMIT 5").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,country_user,text_cleaned_lemma,text,similarity
0,Netherlands,tap john hewitt belfast hazy blonde coloured b...,"Tap at John Hewitt, Belfast: Hazy blonde colou...",0.758187
1,United Kingdom,bottle vinyl taste edinburgh pour amber know h...,"Bottle @All My Vinyl Tasting, Edinburgh. Pours...",0.749002
2,Germany,bottle ms mahogony tan head aroma toast malt b...,Bottle from M&S. Mahogony with a tan head. Aro...,0.732442
3,United Kingdom,burny smoky taste ipa flavour slightly dissapo...,Burny smoky taste to this no ipa flavour what ...,0.636238
4,United Kingdom,bottle home belfast beer festivaldark chestnut...,Bottle at home from Belfast beer festivalDark ...,0.679432


In [91]:
# Calculates for each country, the top reviews based on their similarity.
con.execute("""
CREATE OR REPLACE VIEW top_reviews AS (
    SELECT
        country_user,
        text_cleaned_lemma,
        text,
        similarity,
        ROW_NUMBER() OVER (PARTITION BY country_user ORDER BY similarity DESC) as rn
    FROM review_similarities
)
""")

In [6]:
# Calculates for each country, the most similar representative reviews with other countries and the most representative review for this country.
con.execute("""
DROP VIEW IF EXISTS final_output;
DROP TABLE IF EXISTS final_output;
CREATE TABLE final_output AS (
    WITH similar_countries AS (
        SELECT
            country1,
            MAX(CASE WHEN rn = 1 THEN country2 END) as similar_country1,
            MAX(CASE WHEN rn = 1 THEN ROUND(similarity::numeric, 3) END) as similarity1,
            MAX(CASE WHEN rn = 2 THEN country2 END) as similar_country2,
            MAX(CASE WHEN rn = 2 THEN ROUND(similarity::numeric, 3) END) as similarity2,
            MAX(CASE WHEN rn = 3 THEN country2 END) as similar_country3,
            MAX(CASE WHEN rn = 3 THEN ROUND(similarity::numeric, 3) END) as similarity3
        FROM top_similar_countries
        WHERE rn <= 3
        GROUP BY country1
    ),
    representative_reviews AS (
        SELECT
            country_user,
            MAX(CASE WHEN rn = 1 THEN text END) as review1,
            MAX(CASE WHEN rn = 1 THEN ROUND(similarity::numeric, 3) END) as review_similarity1,
            MAX(CASE WHEN rn = 2 THEN text END) as review2,
            MAX(CASE WHEN rn = 2 THEN ROUND(similarity::numeric, 3) END) as review_similarity2,
            MAX(CASE WHEN rn = 3 THEN text END) as review3,
            MAX(CASE WHEN rn = 3 THEN ROUND(similarity::numeric, 3) END) as review_similarity3
        FROM top_reviews
        WHERE rn <= 3
        GROUP BY country_user
    )
    SELECT
        s.country1,
        s.similar_country1,
        s.similarity1,
        s.similar_country2,
        s.similarity2,
        s.similar_country3,
        s.similarity3,
        r.review1,
        r.review_similarity1,
        r.review2,
        r.review_similarity2,
        r.review3,
        r.review_similarity3
    FROM similar_countries s
    LEFT JOIN representative_reviews r ON s.country1 = r.country_user
    ORDER BY s.country1
);
""")

In [93]:
con.execute("SELECT * FROM final_output LIMIT 5").fetchdf()

,country1,similar_country1,similarity1,similar_country2,similarity2,similar_country3,similarity3,review1,review_similarity1,review2,review_similarity2,review3,review_similarity3
0,Abkhazia,Dominican Republic,0.903,Marshall Islands,0.903,Germany,0.899,"All that we search in Porterâs: Chocolate, b...",0.829,You should try better beers than Katzerer (mic...,0.817,Is really hard brew a beer like Apple Ephemere...,0.752
1,Afghanistan,Panama,0.984,Brazil,0.983,El Salvador,0.983,Definitely by far the best Thai made beer sold...,0.859,grapefruit/nectarine turning resiny on nosing....,0.835,God in heaven save me from this muck. Its not ...,0.832
2,Albania,Slovakia,0.918,Montenegro,0.915,South Korea,0.914,This is maybe one of the most famous pilseners...,0.807,Itâs better than the original Birra Tirana ...,0.804,better than coors light for the cost.. : 0 the...,0.750
3,Algeria,Faroe Islands,0.984,Denmark,0.983,Luxembourg,0.983,Chestnut brown with full bodied foam. Spices -...,0.873,Nice Beer made With a Nice rounded taste of ca...,0.868,"Rich tasting ale with notes of caramel, malt, ...",0.853
4,American Samoa,Fiji Islands,0.636,Honduras,0.635,Saudi Arabia,0.631,i have never tried this beer but my great 10th...,1.000,None,NaN,None,NaN


In [96]:
con.execute("SELECT * FROM final_output WHERE country1 = 'Mexico'").fetchdf()

,country1,similar_country1,similarity1,similar_country2,similarity2,similar_country3,similarity3,review1,review_similarity1,review2,review_similarity2,review3,review_similarity3
0,Mexico,El Salvador,0.999,Taiwan,0.999,Hong Kong,0.999,My first english strong ale thanks to Duff. I ...,0.901,Heritage vintage 2013.APARIENCIA: De color mar...,0.889,"Bottle, 500ml. Pours a clear golden colour wit...",0.887


In [29]:
# Create the JSON files used on the website.
def jsonize(df):
    country = df['country1'].iloc[0]
    similar_countries = []
    for i in range(1, 4):
        similar_countries.append({
            "country": df[f"similar_country{i}"].iloc[0],
            "similarity": df[f"similarity{i}"].iloc[0]
        })

    reviews = []
    for i in range(1, 4):
        if df[f"review{i}"].iloc[0] is None:
            # If the country has less than 3 reviews, ensure no nulls.
            continue

        reviews.append({
            "text": df[f"review{i}"].iloc[0],
            "similarity": df[f"review_similarity{i}"].iloc[0]
        })

    return {
        "country": country,
        "similar_countries": similar_countries,
        "reviews": reviews
    }

In [26]:
jsonize(con.execute("SELECT * FROM final_output WHERE country1 = 'Mexico'").fetchdf())

{'country': 'Mexico',
 'similar_countries': [{'country': 'El Salvador', 'similarity': 0.999},
  {'country': 'Taiwan', 'similarity': 0.999},
  {'country': 'Hong Kong', 'similarity': 0.999}],
 'reviews': [{'text': 'My first english strong ale thanks to Duff. I liked the bottle, very classic looking. Very nice aroma, like dubbel mixed with an english bitter. Opaque and brown pour. Sweet and strongly alcoholic tasting at first, dark fruits, leather and some roastiness in the middle...minimal hoppiness. Finishes with a slight vegetable, cinnamon and sweet malt folowed by a bit of a burn. It did not go well with my hamburger buy i would guess some lamb and chutney would do it grace.',
   'similarity': 0.901},
  {'text': 'Heritage vintage 2013.APARIENCIA: De color marrÃ³n muy oscuro, no llega a ser negra totalmente, de corona de espuma color beige, de burbuja compacta de muy buena retenciÃ³n.AROMA: muy complejo, la notas del roble estÃ¡n presentes, se llega a inhalar el alcohol, notas de frut

In [31]:
# Store all JSON files.
import json
for country in countries:
    if country is None:
        continue
    data = jsonize(con.execute(f"SELECT * FROM final_output WHERE country1 = '{country}'").fetchdf())
    country = countriaze(country)
    with open(DATA_OUT + country + '/output.json', 'w') as f:
        f.write(json.dumps(data))